In [1]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
# Function to load the dataset
class CustomDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.label_encoder = LabelEncoder()
        self.images, self.labels = self.load_dataset()

    def load_dataset(self):
        images = []
        labels = []
        label_names = os.listdir(self.data_dir)
        
        for label in label_names:
            label_dir = os.path.join(self.data_dir, label)
            if os.path.isdir(label_dir):
                for img_name in os.listdir(label_dir):
                    img_path = os.path.join(label_dir, img_name)
                    # Read the image
                    image = cv2.imread(img_path)
                    if image is not None:
                        # Resize the image to the fixed size
                        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
                        if self.transform:
                            image = self.transform(image)
                        images.append(image)
                        labels.append(label)
        
        # Encode labels
        labels = self.label_encoder.fit_transform(labels)
        labels = torch.tensor(labels, dtype=torch.long)
        
        return images, labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        return image, label

# Specify the directory where your dataset is located
data_dir = 'AugmentedAlzheimerDataset'

# Define transformations (resize and normalize)
data_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Example normalization
])

# Load the dataset
dataset = CustomDataset(data_dir, transform=data_transform)

# Split the dataset into training and testing sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [3]:
# Define the CNN model
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(128 * 16 * 16, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# Instantiate the model
model = CNN(num_classes=len(dataset.label_encoder.classes_))

In [4]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [5]:
# Training loop
num_epochs = 20
model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')

Epoch [1/20], Loss: 1.0726
Epoch [2/20], Loss: 0.7870
Epoch [3/20], Loss: 0.7037
Epoch [4/20], Loss: 0.6339
Epoch [5/20], Loss: 0.5857
Epoch [6/20], Loss: 0.5490
Epoch [7/20], Loss: 0.5262
Epoch [8/20], Loss: 0.5028
Epoch [9/20], Loss: 0.4856
Epoch [10/20], Loss: 0.4663
Epoch [11/20], Loss: 0.4481
Epoch [12/20], Loss: 0.4338
Epoch [13/20], Loss: 0.4247
Epoch [14/20], Loss: 0.4051
Epoch [15/20], Loss: 0.4005
Epoch [16/20], Loss: 0.3921
Epoch [17/20], Loss: 0.3815
Epoch [18/20], Loss: 0.3738
Epoch [19/20], Loss: 0.3670
Epoch [20/20], Loss: 0.3598


In [6]:
# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Accuracy: {(100 * correct / total):.2f}%')

Test Accuracy: 86.60%


In [7]:
# Save the model
torch.save(model.state_dict(), 'cnn_model.pth')